In [1]:
import os
import numpy as np
import pandas as pd

from tqdm import tqdm
from google.cloud import storage

In [2]:
fd = "/home/jupyter/ai_font/data/test/플레이브밤비"

In [3]:
fonts = np.unique([f.split("__")[0] for f in os.listdir(fd) if f.endswith(".png")])

In [4]:
def get_all_korean():

    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()

    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns

In [5]:
def get_lexicon(letter):
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2
    return np.array([ch1, ch2, ch3])

In [6]:
ak = get_all_korean()

In [7]:
lex_mapper = {}
for k in tqdm(ak):
    lex_mapper[k] = {
        'double': [],
        'single': [],
    }
    

100%|██████████| 11172/11172 [00:00<00:00, 1064246.29it/s]


In [8]:
i = 0
for k in tqdm(ak):
    lex_k = get_lexicon(k)
    i += 1
    for l in ak[i:]:
        if not k == l:
            lex_l = get_lexicon(l)
            if np.sum(lex_k == lex_l) == 2:
                lex_mapper[k]['double'].append(l)
                lex_mapper[l]['double'].append(k)
            elif np.sum(lex_k == lex_l) == 1:
                lex_mapper[k]['single'].append(l)
                lex_mapper[l]['single'].append(k)

100%|██████████| 11172/11172 [10:46<00:00, 17.29it/s] 


In [20]:
testmap = {}
for font in fonts:
    testmap[font] = {}
    for k in ak:
        testmap[font][k] = {
            'double':set(),
            'single':set(),
        }

In [21]:
for f in tqdm(os.listdir(testfd)):
    if f.endswith(".png"):
        font, letter = f.split(".")[0].split("__")
        for k in lex_mapper[letter]['double']:
            testmap[font][k]['double'].add(letter)
        for k in lex_mapper[letter]['single']:
            testmap[font][k]['single'].add(letter)

100%|██████████| 115322/115322 [01:54<00:00, 1009.02it/s]


In [26]:
testdf = []
for font in tqdm(fonts):
    for l in ak:
        testdf.append({
            'font': font,
            'letter': l,
            "double": list(testmap[font][l]['double']),
            "single": list(testmap[font][l]['single']),
        })

100%|██████████| 16/16 [00:33<00:00,  2.10s/it]


In [27]:
pd.DataFrame().from_dict(testdf).to_pickle("/home/jupyter/ai_font/data/test/testmapdf.pickle")

In [2]:
testdf = pd.read_pickle("/home/jupyter/ai_font/data/test/testmapdf.pickle")

In [5]:
testdf.iloc[0]

font                                                 어비 나현체
letter                                                    가
double    [각, 자, 바, 값, 갇, 마, 긔, 갈, 짜, 교, 따, 갛, 하, 갚, 타, ...
single    [쥐, 탄, 쳬, 재, 표, 수, 뷰, 겅, 결, 경, 삽, 쪼, 걸, 깟, 냐, ...
Name: 0, dtype: object

In [ ]:
닠